### combine a list of SA files into a single one for further analysis
#### the first file defines the wavelength space onto which the other files are interpolated

In [17]:
import pandas as pd
from scipy.interpolate import interp1d

In [18]:
path1 = '/Users/jpw/Analysis/NIRSPEC/iSHELL/240107/rectified/'
path2 = path1
name1 = 'HLTau_PA228_1_SA.csv'
name2 = 'HLTau_PA228_2_SA.csv'
files = [path1+name1, path2+name2]

In [19]:
t0 = pd.read_csv(files[0], skipinitialspace=True)
wl0 = t0['wavelength']
nwl0 = wl0.size
SAneg0 = t0['off_neg']
SApos0 = t0['off_pos']
SAneg_err0 = t0['err_neg']
SApos_err0 = t0['err_pos']
columns = t0.columns[1:]

In [20]:
# add new columns to the master table (this is kind of cool but overly complicated way to combine the data...)
#for n, f in enumerate(files[1:]):
#    t = pd.read_csv(f, skipinitialspace=True)
#    wl = t['wavelength']
#    for c in columns:
#        t0.insert(t0.columns.size, c+str(n+1), interp1d(wl, t[c], bounds_error=False)(wl0))

In [45]:
# add two arrays with and not or logic on the nan
def add_ignore_nan(x, y):
    z = []
    for i in range(x.size):
        x1 = x[i]
        y1 = y[i]
        if np.isfinite(x1) or np.isfinite(y1):
            z.append(np.nansum([x1, y1]))
        else:
            z.append(np.nan)
    return np.array(z)

In [38]:
# note that the errors in the files are variance so the weights are 1/err
# the weighted mean of the error is then Sigma (1) / Sigma (1/err)
SAneg_c = SAneg0 / SAneg_err0
SApos_c = SApos0 / SApos_err0
Wneg_c = 1 / SAneg_err0
Wpos_c = 1 / SApos_err0

for f in files[1:]:
    t = pd.read_csv(f, skipinitialspace=True)
    wl = t['wavelength']
    SAneg = interp1d(wl, t['off_neg'], bounds_error=False)(wl0)
    SApos = interp1d(wl, t['off_pos'], bounds_error=False)(wl0)
    SAneg_err = interp1d(wl, t['err_neg'], bounds_error=False)(wl0)
    SApos_err = interp1d(wl, t['err_pos'], bounds_error=False)(wl0)
    
    SAneg_c = add_ignore_nan(SAneg_c, SAneg / SAneg_err)
    SApos_c = add_ignore_nan(SApos_c, SApos / SApos_err)
    Wneg_c = add_ignore_nan(Wneg_c, 1 / SAneg_err)
    Wpos_c = add_ignore_nan(Wpos_c, 1 / SApos_err)

n = len(files)
SAneg_c /= Wneg_c
SApos_c /= Wpos_c
SAneg_err_c = n / Wneg_c
SApos_err_c = n / Wpos_c

In [40]:
# write out the combined file
csvfile = open('combined_SA.csv', 'w')
csvfile.write('wavelength,  off_neg, off_pos, err_neg, err_pos\n')
               
for i in range(wl0.size):
    csvfile.write(f'{wl0[i]:11.9f}, {SAneg_c[i]:7.4f}, {SApos_c[i]:7.4f}, {SAneg_err_c[i]:7.4f}, {SApos_err_c[i]:7.4f}\n')

    
csvfile.close()